In [ ]:
pip install talos

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.4/56.4 KB 2.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.4/5.4 MB 10.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 52.3/52.3 KB 2.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 26.4/26.4 MB 10.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 16.3 MB/s eta 0:00:00
  Created wheel for chances: filename=chances-0.1.9-py3-none-any.whl size=41606 sha256=ddf067cb97178544a334f440455effbb553ea9d5dc538a170428b93f7a9f7c5b
  Stored in directory: /root/.cache/pip/wheels/c3/26/2b/8ae098b8e9dd7cc1fd0a9131c0f1efea60d710ff7906bf0ab4
  Created wheel for sklearn: filename=sklearn-0.0.post1-py3-none-any.whl size=2955 sha256=7995d3b0b8270343dfc8cd5a1e16822ec7f1d4559dfd9ef54098aa562ebb6add
  Stor

In [ ]:
# mount Google drive
from google.colab import drive

drive.mount('/content/gdrive', force_remount=True)

Mounted at /content/gdrive


In [ ]:
cd gdrive/MyDrive/CPSC5440/Assignment2


/content/gdrive/MyDrive/CPSC5440/Assignment2


In [ ]:
# import libraries
import talos
import tensorflow as tf
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense, Dropout, Flatten
from keras.datasets import cifar100
from tensorflow.keras.datasets import mnist
from tensorflow.keras import backend
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
# import cifar100 data with train/test split
img_rows, img_cols = 32, 32

# the data, split between train and test sets
#(x_train, y_train), (x_test, y_test) = mnist.load_data()
(x_train, y_train), (x_test, y_test) = cifar100.load_data()

if backend.image_data_format() == 'channels_first':
    x_train = x_train.reshape(x_train.shape[0], 3, img_rows, img_cols)
    x_test = x_test.reshape(x_test.shape[0], 3, img_rows, img_cols)
    input_shape = (1, img_rows, img_cols)
else:
    x_train = x_train.reshape(x_train.shape[0], img_rows, img_cols, 3)
    x_test = x_test.reshape(x_test.shape[0], img_rows, img_cols, 3)
    input_shape = (img_rows, img_cols, 1)

x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255
x_test /= 255
print('x_train shape:', x_train.shape)
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')

# convert class vectors to binary class matrices
y_train = tf.keras.utils.to_categorical(y_train, 100)
y_test = tf.keras.utils.to_categorical(y_test, 100)



169001437/169001437 [==============================] - 2s 0us/step
x_train shape: (50000, 32, 32, 3)
50000 train samples
10000 test samples


In [ ]:
# build the model
p = {
    'units': [120, 240],
    'hidden_activations': ['relu', 'sigmoid'],
    'activation': ['softmax', 'sigmoid'],
    'loss': ['mse', 'categorical_crossentropy'],
    'optimizer': ['adam', 'adagrad'],
    'batch_size': [1000, 2000]
}

def my_model(x_train, y_train, x_val, y_val, params):

    model = Sequential()
    model.add(Flatten(input_shape=(32, 32, 3)))

    unitwidth = params['units']-100
    model.add(Dense(units=100 + unitwidth))
    model.add(Dense(units=100 + unitwidth*80//100, activation=params['hidden_activations']))
    model.add(Dense(units=100 + unitwidth*60//100, activation=params['hidden_activations']))
    model.add(Dense(units=100 + unitwidth*40//100, activation=params['hidden_activations']))
    model.add(Dense(units=100 + unitwidth*20//100, activation=params['hidden_activations']))
    model.add(Dense(units=100 + unitwidth*10//100, activation=params['hidden_activations']))
    model.add(Dense(units=100, activation=params['activation']))
    model.compile(loss=params['loss'],
                  optimizer=params['optimizer'],
                  metrics=['accuracy'])

    out = model.fit(x_train, y_train,
                    validation_data=(x_val, y_val),
                    batch_size=params['batch_size'],
                    epochs=200,
                    verbose=0)
    return out, model



In [ ]:
# train and test the model
talos.Scan(x_train, y_train, p, my_model, x_val=x_test, y_val=y_test, experiment_name="talos_output")

In [ ]:
# dispaly comparison table 
r = talos.Reporting('talos_output/report1.csv')
r.table("accuracy")


,hidden_activations,loss,accuracy,round_epochs,activation,val_loss,loss.1,optimizer,val_accuracy,units,batch_size
44,relu,0.628213,0.81246,200,sigmoid,11.164866,categorical_crossentropy,adam,0.1887,240,1000
36,relu,0.661659,0.80184,200,softmax,10.638926,categorical_crossentropy,adam,0.1956,240,1000
37,relu,0.780039,0.78292,200,softmax,8.568357,categorical_crossentropy,adam,0.2022,240,2000
45,relu,0.927586,0.74498,200,sigmoid,8.159774,categorical_crossentropy,adam,0.1985,240,2000
33,relu,0.004373,0.64406,200,softmax,0.011069,mse,adam,0.1979,240,2000
...,...,...,...,...,...,...,...,...,...,...,...
34,relu,0.009903,0.00802,200,softmax,0.009903,mse,adagrad,0.0080,240,1000
62,sigmoid,4.605223,0.00794,200,sigmoid,4.605175,categorical_crossentropy,adagrad,0.0080,240,1000
54,sigmoid,4.605205,0.00760,200,softmax,4.605167,categorical_crossentropy,adagrad,0.0090,240,1000
43,relu,0.010713,0.00758,200,sigmoid,0.010678,mse,adagrad,0.0080,240,2000
